## 画像認識

データ前処理、データセットの読み込み・加工

In [ ]:
from PIL import Image
import numpy as np
import os
import glob
import re
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation,Dropout,Flatten,Dense,GlobalAveragePooling2D
from keras.utils  import np_utils


# クラスラベル
labels = ["grape","apple","orange"]
# ディレクトリ
dataset_dir = "/IR/data/dataset.npy" # 前処理済みデータ
model_dir   = "/IR/data/cnn_h5"      # 学習済みモデル
# リサイズ設定
resize_settings = (50,50)

# 画像データ
X_train = [] # 学習
y_train = [] # 学習ラベル
X_test  = [] # テスト
y_test  = [] # テストラベル


In [ ]:
for class_num, label in enumerate(labels):
    
    # 写真のディレクトリ
    photos_dir = "/IR/data/" + label
    
    # 画像データを取得
    files = glob.glob(photos_dir + "/*.jpg")
   
    #写真を順番に取得
    for i,file in enumerate(files):
        
        # 画像を1つ読込
        image = Image.open(file)
        
        # 画像をRGBの3色に変換
        image = image.convert("RGB")
        
        # 画像のサイズを揃える
        image = image.resize(resize_settings)
        
        # 画像を数字の配列変換
        data  = np.asarray(image) 

        # テストデータ追加
        if i == rand - 1:
            X_test.append(data)
            y_test.append(class_num)
        
        # 学習データかさ増し
        else:            
            # -25度から20度まで5度刻みで回転したデータを追加
            for angle in range(-25,20,5):
                # 回転
                img_r = image.rotate(angle)
                # 画像 → 数字の配列変換
                data  = np.asarray(img_r)
                # 追加
                X_train.append(data)
                y_train.append(class_num)
                # 画像左右反転
                img_trans = image.transpose(Image.FLIP_LEFT_RIGHT)
                data      = np.asarray(img_trans)
                # 追加
                X_train.append(data)
                y_train.append(class_num)        


# X,YがリストなのでTensorflowが扱いやすいようnumpyの配列に変換
X_train = np.array(X_train)
X_test  = np.array(X_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)


# 前処理済みデータを保存
dataset = (X_train,X_test,y_train,y_test)
np.save(dataset_dir,dataset)

/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


#### モデル学習・評価 | CNN

In [ ]:
# メインの関数定義
def main():
    """
    ①データの前処理(エンコーディング)
    """
    # 保存したnumpyデータ読み込み
    X_train,X_test,y_train,y_test = np.load(dataset_dir, allow_pickle=True)
    
    # 0~255の整数範囲になっているため、0~1間に数値が収まるよう正規化
    X_train = X_train.astype("float") / X_train.max()
    X_test  = X_test.astype("float") /  X_train.max()
    
    # クラスラベルの正解値は1、他は0になるようワンホット表現を適用
    y_train = np_utils.to_categorical(y_train,len(labels))
    y_test  = np_utils.to_categorical(y_test,len(labels))
    """
    ②モデル学習&評価
    """
    #モデル学習
    model = model_train(X_train,y_train)
    
    #モデル評価
    evaluate(model,X_test, y_test)
    
  
    
#モデル学習関数
def model_train(X_train,y_train):
    
    #インスタンス
    model = Sequential()
    
    # 1層目 (畳み込み）
    model.add(Conv2D(32,(3,3),padding="same", input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))

    # 2層目 (Max Pooling)
    model.add(MaxPooling2D(pool_size=(2,2)))                     
    model.add(Dropout(0.3))         

    # 3層目 (畳み込み)
    model.add(Conv2D(64,(3,3),padding="same"))                   
    model.add(Activation('relu'))

    # 4層目 (Max Pooling)
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Global Average Pooling
    model.add(GlobalAveragePooling2D())

    # 5層目 (全結合層)
    model.add(Dense(512))                                       
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # 出力層(softmaxで0〜1の確率を返す)
    model.add(Dense(3)) 
    model.add(Activation('softmax'))

    # 最適化アルゴリズム Adam
    opt = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)

    # 損失関数
    model.compile(loss="categorical_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy"]
                 )
                  
    # モデル学習
    model.fit(X_train,y_train,batch_size=10,epochs=150)
    
    # モデルの結果を保存
    model.save(model_dir)
    return model
    

# 評価用関数
def evaluate(model,X_test,y_test):
    # モデル評価
    scores = model.evaluate(X_test,y_test,verbose=1)
    print("Test Loss: ", scores[0])
    print("test Accuracy: ", scores[1])

In [ ]:
# モデル学習実行
model = main()

FileNotFoundError: ignored